In [1]:
import os
import h5py
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plot
import numpy as np
import json
import io
import unicodecsv as csv
import tensorflow as tf
import gc
#import PIL
#from PIL import Image
#print('Pillow Version:', PIL.__version__)

import numpy as np

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, LSTM
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
#from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers



import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


import numpy as np
import os
import sys
import time
import pandas as pd 
#from tqdm._tqdm_notebook import tqdm_notebook
import pickle

cutoff = 10.0

# from keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#import logging


/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
INPUT_PATH = "/home/aj/eurusd"
TIMESTEPS = 100
BATCHSIZE = 100
NUMRECORDS = 10398
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
eurusddata = pd.read_csv(os.path.join(INPUT_PATH, "EURUSDDAILY.csv"), engine='python')
print(eurusddata.shape)
print(eurusddata.columns)

#display(eurusddata.head(5))
#display(eurusddata.tail(1451))



dim_0 = eurusddata.shape[0] - TIMESTEPS 
dim_1 = eurusddata.shape[1] -2
print(dim_0)
print(dim_1)
dim_1 = 1

(10398, 6)
Index(['Date', 'Price', 'Open', 'High', 'Low', 'Change %'], dtype='object')
10298
4


In [4]:
#ffg
pricearray = np.zeros((10398,1))

#gg
eurusddata1 = eurusddata.iloc[::-1]
display(eurusddata1.head(1451))
#display(eurusddata1.tail(5))


# IMPORTANT: Data flat from record# 0 till record# 1450



,Date,Price,Open,High,Low,Change %
10397,8-Aug-80,1.4599,1.4599,1.4599,1.4599,-0.08%
10396,11-Aug-80,1.4592,1.4592,1.4592,1.4592,-0.05%
10395,12-Aug-80,1.4607,1.4607,1.4607,1.4607,0.10%
10394,13-Aug-80,1.4620,1.4620,1.4620,1.4620,0.09%
10393,14-Aug-80,1.4571,1.4571,1.4571,1.4571,-0.34%
...,...,...,...,...,...,...
8951,30-Apr-86,0.9938,0.9938,0.9938,0.9938,0.09%
8950,1-May-86,0.9793,0.9793,0.9793,0.9793,-1.46%
8949,2-May-86,0.9790,0.9790,0.9790,0.9790,-0.03%
8948,6-May-86,0.9747,0.9747,0.9747,0.9747,-0.44%


        
for i in range(dim_0+TIMESTEPS):
            
            #print(TIMESTEPS+j+1)
        pricearray[i] = eurusddata1.iloc[0+i, 3:4]

In [5]:
for i in range(dim_0+TIMESTEPS):
            
            #print(TIMESTEPS+j+1)
        pricearray[i] = eurusddata1.iloc[0+i, 4:5]

In [6]:

print(pricearray[1449])
offsetmin = np.amin(pricearray)
maxinrange = np.amax(pricearray)
print(offsetmin)
print(maxinrange)

[0.9747]
0.6444
1.5865


In [7]:
print(pricearray[5000])
print(pricearray[5100])
pricearray = pricearray/(maxinrange + 0.0001)
print(pricearray[5000])
print(pricearray[5100])

[1.0045]
[0.8845]
[0.63311484]
[0.55748141]


In [8]:
offsetmin = np.amin(pricearray)
maxinrange = np.amax(pricearray)
print(offsetmin)
print(maxinrange)

0.40615151897138535
0.9999369721416866


In [9]:
#for i in range(len(pricearray)):
   # if pricearray[i] == 0.0:
     #   print(i)

In [10]:
#fsdfsa

In [11]:
x = np.zeros((dim_0-1450, 1, TIMESTEPS ))
y = np.zeros((dim_0-1450, dim_1))

In [12]:
x.shape


(8848, 1, 100)

In [13]:
y.shape

(8848, 1)

In [14]:
'''
arr = eurusddata1.to_numpy()
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [120, 80]
x = np.arange(10398)

y1 = arr[:, 1]
y2 = arr[:, 2]
y3 = arr[:, 3]
y4 = arr[:, 4]
print(y1)
print(y2)
print(y3)
print(y4)


#plt.plot(x, y1, label="John")

#plt.plot(x, y2, label="David")

#plt.plot(x, y3, label="Tom")

plt.plot(x, y3 - y4, label="Tomato")
'''
#dsada
eurusddata1 = eurusddata.iloc[::-1]
#display(eurusddata1.head(61))
#display(eurusddata1.tail(5))

for i in range(dim_0-1450):
            
            #print(TIMESTEPS+j+1)
            s = pricearray[1450+i:1450+TIMESTEPS+i, :]
            x[i] = s.reshape(1,TIMESTEPS)
            y[i] = pricearray[1450+TIMESTEPS+i]

In [15]:


print(x[0])
print(y[0])

print(x[1])
print(y[1])

print(x[10296-1450])
print(y[10296-1450])

print(x[10297-1450])
print(y[10297-1450])


[[0.61817723 0.61792512 0.62032018 0.61370226 0.6140174  0.61464767
  0.60866003 0.60594983 0.60122274 0.60059246 0.59460482 0.59132737
  0.59315517 0.58987773 0.59347031 0.58924745 0.58300769 0.57853271
  0.58660028 0.59347031 0.59763015 0.60386991 0.60147485 0.6009076
  0.60929031 0.60929031 0.6074625  0.61225262 0.60450019 0.60147485
  0.6036178  0.6009076  0.59372243 0.60450019 0.60689525 0.60689525
  0.60834489 0.61256775 0.61672759 0.61553006 0.62000504 0.62032018
  0.61912265 0.61792512 0.61464767 0.61672759 0.61521492 0.61137023
  0.61641245 0.62032018 0.62176982 0.62032018 0.62479516 0.62384974
  0.62088743 0.62416488 0.61937476 0.62176982 0.62655994 0.62775747
  0.63286273 0.63494264 0.63462751 0.6316652  0.63614017 0.63941762
  0.6373377  0.64181268 0.6384722  0.63941762 0.64237993 0.63998487
  0.64030001 0.63878734 0.64534224 0.64357746 0.64565738 0.64565738
  0.64445985 0.64149754 0.64294718 0.64237993 0.64981722 0.64956511
  0.64748519 0.64981722 0.64653977 0.63878734 0.6

In [16]:
'''
x1 = x[:,:,1]
y1 = y[:,:,1]
print(x1)
print(y1)
'''

'\nx1 = x[:,:,1]\ny1 = y[:,:,1]\nprint(x1)\nprint(y1)\n'

In [17]:
'''
x = x1 - x0
y = y1 - y0
for i in range(1):
           print(x[i])
           print(y[i])

j=1

print(x[3015])
print(y[3015])
x =  x[3015:,:]
y =  y[3015:,:]
'''


'\nx = x1 - x0\ny = y1 - y0\nfor i in range(1):\n           print(x[i])\n           print(y[i])\n\nj=1\n\nprint(x[3015])\nprint(y[3015])\nx =  x[3015:,:]\ny =  y[3015:,:]\n'

In [18]:
x.shape

(8848, 1, 100)

In [19]:
y.shape

(8848, 1)

In [20]:
for i in range(5):
    print(x[i])
    print(y[i])

[[0.61817723 0.61792512 0.62032018 0.61370226 0.6140174  0.61464767
  0.60866003 0.60594983 0.60122274 0.60059246 0.59460482 0.59132737
  0.59315517 0.58987773 0.59347031 0.58924745 0.58300769 0.57853271
  0.58660028 0.59347031 0.59763015 0.60386991 0.60147485 0.6009076
  0.60929031 0.60929031 0.6074625  0.61225262 0.60450019 0.60147485
  0.6036178  0.6009076  0.59372243 0.60450019 0.60689525 0.60689525
  0.60834489 0.61256775 0.61672759 0.61553006 0.62000504 0.62032018
  0.61912265 0.61792512 0.61464767 0.61672759 0.61521492 0.61137023
  0.61641245 0.62032018 0.62176982 0.62032018 0.62479516 0.62384974
  0.62088743 0.62416488 0.61937476 0.62176982 0.62655994 0.62775747
  0.63286273 0.63494264 0.63462751 0.6316652  0.63614017 0.63941762
  0.6373377  0.64181268 0.6384722  0.63941762 0.64237993 0.63998487
  0.64030001 0.63878734 0.64534224 0.64357746 0.64565738 0.64565738
  0.64445985 0.64149754 0.64294718 0.64237993 0.64981722 0.64956511
  0.64748519 0.64981722 0.64653977 0.63878734 0.6

In [21]:
arr_rand = np.random.rand(x.shape[0])
split = arr_rand < np.percentile(arr_rand, 90)

x_train = x[split]
y_train = y[split]
x_testeval = x[~split]
y_testeval = y[~split]

print(x_train.shape)
print(y_train.shape)
print(x_testeval.shape) 
print(y_testeval.shape)

arr_rand1 = np.random.rand(x_testeval.shape[0])
split1 = arr_rand1 < np.percentile(arr_rand1, 90)

x_test = x_testeval[split1]
y_test = y_testeval[split1]
x_eval = x_testeval[~split1]
y_eval = y_testeval[~split1]

print(x_test.shape) 
print(y_test.shape)
print(x_eval.shape) 
print(y_eval.shape)



(7963, 1, 100)
(7963, 1)
(885, 1, 100)
(885, 1)
(796, 1, 100)
(796, 1)
(89, 1, 100)
(89, 1)


In [22]:
for i in range(5):
    print(x_test[i])
    print(y_test[i])

[[0.60386991 0.60147485 0.6009076  0.60929031 0.60929031 0.6074625
  0.61225262 0.60450019 0.60147485 0.6036178  0.6009076  0.59372243
  0.60450019 0.60689525 0.60689525 0.60834489 0.61256775 0.61672759
  0.61553006 0.62000504 0.62032018 0.61912265 0.61792512 0.61464767
  0.61672759 0.61521492 0.61137023 0.61641245 0.62032018 0.62176982
  0.62032018 0.62479516 0.62384974 0.62088743 0.62416488 0.61937476
  0.62176982 0.62655994 0.62775747 0.63286273 0.63494264 0.63462751
  0.6316652  0.63614017 0.63941762 0.6373377  0.64181268 0.6384722
  0.63941762 0.64237993 0.63998487 0.64030001 0.63878734 0.64534224
  0.64357746 0.64565738 0.64565738 0.64445985 0.64149754 0.64294718
  0.64237993 0.64981722 0.64956511 0.64748519 0.64981722 0.64653977
  0.63878734 0.63822009 0.63929157 0.63336695 0.62882894 0.6404891
  0.64250599 0.64313627 0.65328375 0.65662423 0.64313627 0.6404891
  0.6404891  0.6404891  0.63985882 0.64401866 0.64496407 0.64496407
  0.64704399 0.64912391 0.65296861 0.64912391 0.6512

In [23]:
#x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1])) # Reshaping done for LSTM as it needs 3D input
#x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
#x_eval = x_eval.reshape((x_eval.shape[0], 1, x_eval.shape[1]))

In [24]:
#seerwe

In [25]:
def totaltestpredictionerror(y_test, y_testresult):
    ytestdiff = y_test - y_testresult
    ytestdiffabs = np.absolute(ytestdiff)
    sumoftesterrors = np.sum(ytestdiffabs) 
    y_test = None
    y_testresult = None
    ytestdiff = None
    ytestdiffabs =  None
    
    return sumoftesterrors

In [26]:
def totalevalpredictionerror(y_eval, y_evalresult):
    yevaldiff = y_eval - y_evalresult
    yevaldiffabs = np.absolute(yevaldiff)
    sumofevalerrors = np.sum(yevaldiffabs)
    y_eval = None
    y_evaltresult = None
    yevaldiff = None
    ytestdiffabs =  None
    
    return sumofevalerrors

In [27]:
def totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult):
    ytestdiff = y_test - y_testresult
    yevaldiff = y_eval - y_evalresult
    ytestdiffabs = np.absolute(ytestdiff)
    yevaldiffabs = np.absolute(yevaldiff)
    sumoferrors = np.sum(ytestdiffabs) + np.sum(yevaldiffabs)
    y_eval = None
    y_evaltresult = None
    y_test = None
    y_testresult = None
    ytestdiff = None
    yevaldiff = None
    ytestdiffabs =  None
    
    return sumoferrors

In [28]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape) 
print(y_test.shape)
print(x_eval.shape) 
print(y_eval.shape)

(7963, 1, 100)
(7963, 1)
(796, 1, 100)
(796, 1)
(89, 1, 100)
(89, 1)


In [29]:


regressor = Sequential()

regressor.add(LSTM(units = 800, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))

regressor.add(LSTM(units = 800, return_sequences = True))

regressor.add(LSTM(units = 800))

regressor.add(Dense(units = 1))

regressor.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [31]:

with tf.device('gpu'):
    lowfilenumber = 0
    lowscore = 100
    for i in range(60000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028, verbose=0)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)
        

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if lowscore > ytotaldiff:
            lowscore = ytotaldiff
            lowfilenumber = i
            print("lowfilenumber-", lowfilenumber,"                          LOWSCORE - ", lowscore)
        
        cutoff = 4.00
        if ytotaldiff < cutoff:
            print(i+0)
            regressor.save("EUR24H800Xlow"+str(i+0)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            
       

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
lowfilenumber- 2                           LOWSCORE -  31.207948840019096
lowfilenumber- 6                           LOWSCORE -  27.135268784960836
lowfilenumber- 8                           LOWSCORE -  25.723104331677604
lowfilenumber- 9                           LOWSCORE -  25.386713879339545
lowfilenumber- 10                           LOWSCORE -  24.41311081587615
lowfilenumber- 11                           LOWSCORE -  23.598594139266343
lowfilenumber- 12                           LOWSCORE -  22.879821931836464
lowfilenumber- 13                           LOWSCORE -  22.157809325602233
lowfilenumber- 14                           LOWSCORE -  21.473735229741816
lowfilenumber- 15                           LOWSCORE -  20.89180399453251
lowfilenumber- 16                           LOWSCORE -  20.258854195800538
lowfilenumber- 17                           LOWSCORE -  19.718436247498815
lowfilenumbe

lowfilenumber- 754                           LOWSCORE -  5.0135024629038325
lowfilenumber- 759                           LOWSCORE -  4.969059723225695
lowfilenumber- 794                           LOWSCORE -  4.931871134494301
lowfilenumber- 805                           LOWSCORE -  4.913416590942163
lowfilenumber- 833                           LOWSCORE -  4.880956918222823
lowfilenumber- 853                           LOWSCORE -  4.850865496004362
lowfilenumber- 890                           LOWSCORE -  4.8369377943902325
lowfilenumber- 948                           LOWSCORE -  4.791607705563716
lowfilenumber- 970                           LOWSCORE -  4.786515308225891
lowfilenumber- 974                           LOWSCORE -  4.752330172652481
lowfilenumber- 979                           LOWSCORE -  4.741360729126586
lowfilenumber- 988                           LOWSCORE -  4.724115428972046
lowfilenumber- 1006                           LOWSCORE -  4.718098991360201
lowfilenumber- 1052   

2696
  Test --  3.5097459653474843   Eval --  0.4210200369260383   Total -- ------------- 3.9307660022735225
2700
  Test --  3.527042658282726   Eval --  0.43598069912335713   Total -- ------------- 3.963023357406083
2701
  Test --  3.5347532299297315   Eval --  0.4253428610534574   Total -- ------------- 3.960096090983189
lowfilenumber- 2706                           LOWSCORE -  3.8683607474891746
2706
  Test --  3.4460980321230066   Eval --  0.4222627153661679   Total -- ------------- 3.8683607474891746
lowfilenumber- 2707                           LOWSCORE -  3.8679588402969056
2707
  Test --  3.448866305616634   Eval --  0.41909253468027174   Total -- ------------- 3.8679588402969056
2708
  Test --  3.466538522217353   Eval --  0.41850603661410324   Total -- ------------- 3.885044558831456
2720
  Test --  3.5628508160262453   Eval --  0.43576356069476374   Total -- ------------- 3.998614376721009
2727
  Test --  3.4939829281655834   Eval --  0.42663872612389697   Total -- ---------

lowfilenumber- 2980                           LOWSCORE -  3.8214545078931565
2980
  Test --  3.4099848972628557   Eval --  0.41146961063030074   Total -- ------------- 3.8214545078931565
2997
  Test --  3.461001844728006   Eval --  0.4243805085397022   Total -- ------------- 3.8853823532677083
3001
  Test --  3.434769136148608   Eval --  0.414205791880199   Total -- ------------- 3.8489749280288073
3006
  Test --  3.533360698139563   Eval --  0.4181517433296692   Total -- ------------- 3.9515124414692324
3007
  Test --  3.5278968547461353   Eval --  0.42184871435916815   Total -- ------------- 3.9497455691053034
3008
  Test --  3.4440446557019495   Eval --  0.42399548063448955   Total -- ------------- 3.868040136336439
3009
  Test --  3.4212861698617183   Eval --  0.41263115553301033   Total -- ------------- 3.8339173253947285
3015
  Test --  3.4315884555406475   Eval --  0.4107298257070514   Total -- ------------- 3.842318281247699
lowfilenumber- 3016                           LOWSCOR

3201
  Test --  3.4294452738314094   Eval --  0.41418181994095715   Total -- ------------- 3.8436270937723664
3204
  Test --  3.376706381839205   Eval --  0.4045886270684035   Total -- ------------- 3.7812950089076085
3208
  Test --  3.55558190662233   Eval --  0.42577803561685446   Total -- ------------- 3.9813599422391848
3214
  Test --  3.4332139111400677   Eval --  0.4154893822407305   Total -- ------------- 3.8487032933807983
3215
  Test --  3.5405128313225904   Eval --  0.4265430016279018   Total -- ------------- 3.9670558329504924
3218
  Test --  3.455863848604711   Eval --  0.4215203841532321   Total -- ------------- 3.877384232757943
3220
  Test --  3.5009065104074857   Eval --  0.41906791833014234   Total -- ------------- 3.919974428737628
3223
  Test --  3.528636548988879   Eval --  0.42878659734295665   Total -- ------------- 3.9574231463318355
3225
  Test --  3.549354514655828   Eval --  0.42226910498709425   Total -- ------------- 3.971623619642922
3230
  Test --  3.39544

...

6441
  Test --  3.3906422666158145   Eval --  0.39643318773623426   Total -- ------------- 3.7870754543520486
6442
  Test --  3.0019976346200954   Eval --  0.35829955594139584   Total -- ------------- 3.360297190561491
6443
  Test --  3.022367281246858   Eval --  0.3703157441185979   Total -- ------------- 3.392683025365456
6444
  Test --  3.488007780568802   Eval --  0.40499434443480775   Total -- ------------- 3.8930021250036098
6445
  Test --  3.520970905133665   Eval --  0.42884699164220674   Total -- ------------- 3.9498178967758717
6449
  Test --  3.2064575081693256   Eval --  0.3721561277331936   Total -- ------------- 3.5786136359025194
6450
  Test --  3.2170705377011215   Eval --  0.3703622514823194   Total -- ------------- 3.587432789183441
6452
  Test --  3.1167055122656686   Eval --  0.368113067927614   Total -- ------------- 3.4848185801932825
6453
  Test --  3.153395272752877   Eval --  0.378362736552596   Total -- ------------- 3.531758009305473
6454
  Test --  3.0531770

6558
  Test --  3.1376536159741972   Eval --  0.3921181570983209   Total -- ------------- 3.529771773072518
6559
  Test --  2.977235418842884   Eval --  0.3713756713217419   Total -- ------------- 3.3486110901646255
6560
  Test --  2.9954563089491795   Eval --  0.3673568014823275   Total -- ------------- 3.362813110431507
6562
  Test --  3.2225620766898997   Eval --  0.3864962327875522   Total -- ------------- 3.6090583094774518
6564
  Test --  3.0185063879621854   Eval --  0.36510293578855146   Total -- ------------- 3.3836093237507368
6565
  Test --  3.2334408087086017   Eval --  0.4008216057721742   Total -- ------------- 3.634262414480776
6566
  Test --  3.007773852905828   Eval --  0.37431382845167216   Total -- ------------- 3.3820876813575
6567
  Test --  3.3325435650329096   Eval --  0.4092141591659576   Total -- ------------- 3.7417577241988673
6568
  Test --  2.993528966650933   Eval --  0.3536764874398802   Total -- ------------- 3.347205454090813
6569
  Test --  3.064741911

6680
  Test --  3.0274699581656694   Eval --  0.36050138333909787   Total -- ------------- 3.3879713415047674
6681
  Test --  3.437507931783103   Eval --  0.40425403110169345   Total -- ------------- 3.8417619628847968
6682
  Test --  2.9703535172657682   Eval --  0.3619579202918731   Total -- ------------- 3.3323114375576415
6683
  Test --  3.1428731530923577   Eval --  0.3732175931388976   Total -- ------------- 3.516090746231255
6685
  Test --  2.9862062884251657   Eval --  0.3641146091664703   Total -- ------------- 3.350320897591636
6686
  Test --  3.5349169910277105   Eval --  0.4368362628570587   Total -- ------------- 3.9717532538847693
6688
  Test --  3.254065141391744   Eval --  0.3820487041649515   Total -- ------------- 3.6361138455566953
6689
  Test --  3.0303360238797152   Eval --  0.36843669047861105   Total -- ------------- 3.3987727143583264
6692
  Test --  3.0127112298178274   Eval --  0.3581148670688331   Total -- ------------- 3.3708260968866606
6693
  Test --  3.17

6794
  Test --  2.997962198889484   Eval --  0.36033619864206645   Total -- ------------- 3.3582983975315503
6795
  Test --  2.933508138983906   Eval --  0.35527372584131933   Total -- ------------- 3.2887818648252254
6802
  Test --  3.066544410350809   Eval --  0.35694763979905175   Total -- ------------- 3.4234920501498607
6803
  Test --  3.489493313930262   Eval --  0.4033810024482082   Total -- ------------- 3.8928743163784705
6806
  Test --  3.146265233913949   Eval --  0.37010036311413563   Total -- ------------- 3.5163655970280847
6807
  Test --  3.5308763027777226   Eval --  0.4079137297156309   Total -- ------------- 3.9387900324933534
6808
  Test --  3.5126482755993846   Eval --  0.4317690943177944   Total -- ------------- 3.944417369917179
6809
  Test --  3.034092151268153   Eval --  0.37695455263380107   Total -- ------------- 3.411046703901954
6810
  Test --  3.363008979326178   Eval --  0.39682979652413797   Total -- ------------- 3.7598387758503162
6812
  Test --  3.0121

KeyboardInterrupt: 

In [ ]:
bfghfgj
from tensorflow import keras
from tensorflow.keras import layers

regressor = load_model("EURUSD24H800X59228.h5")

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.005,
    decay_steps=1000,
    decay_rate=0.75,
    staircase=True)




In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.002)
regressor.compile(loss='mean_squared_error', optimizer=opt)

In [ ]:
gtc

In [ ]:
with tf.device('gpu'):
    for i in range(75000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028, verbose=0)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print("  Total -- -------------", ytotaldiff)
        cutoff = 3.51
        if ytotaldiff < cutoff:
            print(i+60)
            regressor.save("EURUSD24H"+str(i+60)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 5.2:
            print(i)
            regressor.save("EURUSD24H"+str(i))+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.7:
            print(i+600)
            regressor.save("EURUSD24H"+str(i+600)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.5:
            print(i+1200)
            regressor.save("EURUSD24H"+str(i+1200)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(1200):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.4:
            print(i+1800)
            regressor.save("EURUSD24H"+str(i+1800)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:
with tf.device('gpu'):
    for i in range(3000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.08:
            print(i+3000)
            regressor.save("EURUSD24H"+str(i+3000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:
with tf.device('gpu'):
    for i in range(6000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 3.5348:
            print(i+18000)
            regressor.save("EURUSD24H"+str(i+18000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(20):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
        
        
        cutoff1 = 3.4281
        if ytotaldiff < cutoff1:
            print(" cutoff applied  -- " , cutoff1)
            print(i+60000)
            regressor.save("EURUSD24H"+str(i+60000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            cutoff1 = ytotaldiff
            print("new globval cutoff -- " , cutoff1)
       

In [ ]:
with tf.device('gpu'):
    for i in range(1):
        print(i)
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        regressor.save("EURUSDDAILYCLOSEM7.h5")

In [ ]:
y_testresult = regressor.predict(x_test)
y_evalresult = regressor.predict(x_eval)

In [ ]:
for i in range(5):
    print(y_testresult[i])
    print(y_test[i]) 

In [ ]:
for i in range(5):
    print(y_testresult[i])
    print(y_test[i]) 

In [ ]:
plot.plot(history.history['loss'], label='MAE (training data)')
#plot.plot(history.history['val_loss'], label='MAE (validation data)')

plot.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [120, 80]
x = np.arange(len(y_testresult))
y=y_testresult.reshape(927)
y1=y_test.reshape(927)

yyy = y_testresult - y_test
ydif = yyy.reshape(927)

#plt.plot(x, y, label="John")

#plt.plot(x, y1, label="David")

plt.plot(x, y_testresult - y_test, label="Tom")



In [ ]:
good till here


In [ ]:
scrap
def ResNetFX(input_shape = (1,100)):
    """
    Arguments:
    input_shape -- shape 

    Returns:
    model -- a Model() instance in Keras
    """
    """
    size = 3200
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    X = Dense(size,  activation='relu')(X_input)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(1,  activation='linear')(X)
    """
    
    X_input = Input(input_shape)
    X = LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2]))
    X = Dense(1,  activation='linear')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNetFX')

    return model


In [ ]:
scrapmodel = ResNetFX(input_shape = (100))

'''
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), 
    tf.keras.metrics.Recall(), tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(),
    tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives(), tf.keras.metrics.SensitivityAtSpecificity(),
    tf.keras.metrics.SpecificityAtSensitivity()])
'''
model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mse'])

#model.fit(x_train, y_train, epochs = 10, batch_size = 8)



In [ ]:
scrap
gc.collect()
offset = 0

ytestdiff = np.zeros(600)
yevaldiff = np.zeros(600)
ytotaldiff = np.zeros(600)

#model = load_model("EURUSDDAILY"+str(offset-1)+".h5")
#model.compile(optimizer='adam', loss='mean_squared_error', 
              #metrics=[tf.keras.metrics.MeanAbsoluteError()])

with tf.device('gpu'):
    for i in range(600):
        history = model.fit(x_train, y_train, epochs = 1, batch_size = 64)
        model.save("EURUSD24H"+str(i+offset)+".h5")
        y_testresult = model.predict(x_test)
        y_evalresult = model.predict(x_eval)

        ytestdiff[i] = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff[i] = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff[i] = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)

        print("Iter#--", i, )
        print("  Test -- ", ytestdiff[i], "  Eval -- ", yevaldiff[i], "  Total -- ", ytotaldiff[i])


        gc.collect()
    #hist_df = pd.DataFrame(history.history) 
    # or save to csv: 
    #hist_csv_file = "EURUSDDailyhistory"+str(i+offset)+".csv"
    #with open(hist_csv_file, mode='w') as f:
        #hist_df.to_csv(f)

In [ ]:
scrap
for i in range(50):
    print(y_test[i])
    print(y_testresult[i])